In [0]:
# -*- coding: utf-8 -*-

#@author: alison

import re
import string
import pickle
import keras
import numpy as np
import pandas as pd
from nltk.corpus import stopwords
from sklearn.model_selection import train_test_split
from nltk.stem import PorterStemmer, SnowballStemmer
from nltk.tokenize import TweetTokenizer
from keras.preprocessing.text import Tokenizer
from keras.preprocessing import sequence
from keras.models import Sequential, Model
from keras.layers import Dense, Flatten, Activation
from keras.layers import Conv2D, MaxPool2D, Reshape
from keras.layers import Input, concatenate, Dropout
from keras.layers import Embedding, Concatenate
from keras.optimizers import Adam, SGD, RMSprop
from keras import optimizers
from keras import regularizers

Using TensorFlow backend.


In [0]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [0]:
!pip install -U -q PyDrive
 
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials
 
# 1. Authenticate and create the PyDrive client.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [0]:
file_list = drive.ListFile({'q': "'1JsHac7_mxx_M8BapYSzcDKnzl4M9e1Yb' in parents and trashed=false"}).GetList()
for file1 in file_list:
    print('title: %s, id: %s' % (file1['title'], file1['id']))

title: CNN_en_b.ipynb, id: 1Y4-iz6jsFtExSbQu4Iw0Rdwz23cAcZA2
title: WordEmbedding_BiLSTM_en_b.ipynb, id: 1RsRiCK1r31zWBCi_otXDYl_sCkXPBXgS
title: WordEmbedding_LSTM_en_b.ipynb, id: 1X0gbSY2GSaAFAux8TtspIXHPYefVE01H
title: WordEmbedding_GRU_en_b.ipynb, id: 1f87RaAgRU_RYsrSXZoHMu7ylpMv__0IV
title: bi_lstm_en_b.ipynb, id: 19N0cZ6WKYpnTfDVgKn1i-4UzLEN4jmSK
title: lstm_en_b.ipynb, id: 1n63dDEnPeqMTASzhwnaDnG_8knVqmIkX
title: gru_en_b.ipynb, id: 1l3cd4flv4i1iOwKjLdnDb5082nNkqvYe
title: test_en.tsv, id: 1Z-kJ95PF2VZiGAn_5Piyovk8SZ-bYQJv
title: test_es.tsv, id: 1PlU_aDjY9RLq2CKW1zMQTDmRDcZ0f8YL
title: train_en.tsv, id: 1tv4kRZtCJu7F4WtkBvaMWZMXZZDYF-yv
title: dev_en.tsv, id: 116R1Q_P_m75ZagpkE7cZXfXHiDSDwrJy
title: train_es.tsv, id: 18SENOaqh8YZ98vrzHgHP3sQk3sycMz0H
title: dev_es.tsv, id: 1S5cZDZq7Mhmxpp_SUs79j86An3ICs265


In [0]:
train_downloaded = drive.CreateFile({'id': '1tv4kRZtCJu7F4WtkBvaMWZMXZZDYF-yv'})
train_downloaded.GetContentFile('train_en.tsv')
dev_downloaded = drive.CreateFile({'id': '116R1Q_P_m75ZagpkE7cZXfXHiDSDwrJy'})
dev_downloaded.GetContentFile('dev_en.tsv')
test_downloaded = drive.CreateFile({'id': '1Z-kJ95PF2VZiGAn_5Piyovk8SZ-bYQJv'})
test_downloaded.GetContentFile('test_en.tsv')

In [0]:
train = pd.read_csv('train_en.tsv', delimiter='\t',encoding='utf-8')
dev = pd.read_csv('dev_en.tsv', delimiter='\t',encoding='utf-8')
#test = pd.read_csv('test_en.tsv', delimiter='\t',encoding='utf-8')

In [0]:
# Etapa de pré-processamento

def clean_tweets(tweet):
    tweet = re.sub('@(\\w{1,15})\b', '', tweet)
    tweet = tweet.replace("via ", "")
    tweet = tweet.replace("RT ", "")
    tweet = tweet.lower()
    return tweet
    
def clean_url(tweet):
    tweet = re.sub('http\\S+', '', tweet, flags=re.MULTILINE)   
    return tweet
    
def remove_stop_words(tweet):
    stops = set(stopwords.words("english"))
    stops.update(['.',',','"',"'",'?',':',';','(',')','[',']','{','}'])
    toks = [tok for tok in tweet if not tok in stops and len(tok) >= 3]
    return toks
    
def stemming_tweets(tweet):
    stemmer = SnowballStemmer('english')
    stemmed_words = [stemmer.stem(word) for word in tweet]
    return stemmed_words

def remove_number(tweet):
    newTweet = re.sub('\\d+', '', tweet)
    return newTweet

def remove_hashtags(tweet):
    result = ''

    for word in tweet.split():
        if word.startswith('#') or word.startswith('@'):
            result += word[1:]
            result += ' '
        else:
            result += word
            result += ' '

    return result

In [0]:
def preprocessing(tweet, swords = True, url = True, stemming = True, ctweets = True, number = True, hashtag = True):

    if ctweets:
        tweet = clean_tweets(tweet)

    if url:
        tweet = clean_url(tweet)

    if hashtag:
        tweet = remove_hashtags(tweet)
    
    twtk = TweetTokenizer(strip_handles=True, reduce_len=True)

    if number:
        tweet = remove_number(tweet)
    
    tokens = [w.lower() for w in twtk.tokenize(tweet) if w != "" and w is not None]

    if swords:
        tokens = remove_stop_words(tokens)

    if stemming:
        tokens = stemming_tweets(tokens)

    text = " ".join(tokens)

    return text

In [0]:
train_text  = train['text'].map(lambda x: preprocessing(x, swords = True, url = True, stemming = True, ctweets = True, number = True, hashtag = True))
hs_train    = train['HS']
id_train    = train['id']
tr_train    = train['TR']
ag_train    = [1 if k == '1' else 0 for k in train['AG']]

test_text  = dev['text'].map(lambda x: preprocessing(x, swords = True, url = True, stemming = True, ctweets = True, number = True, hashtag = True))
hs_test    = dev['HS']
id_test    = dev['id']
tr_test    = dev['TR']
ag_test    = dev['AG']

In [0]:
file_list = drive.ListFile({'q': "'1sBKK1i4JXIluelnAPqjOU4xVIIPA3Vmy' in parents and trashed=false"}).GetList()
for file1 in file_list:
    print('title: %s, id: %s' % (file1['title'], file1['id']))

title: GoogleNews-word2vec-300_en.bin, id: 1bERzzLRCcPdavQYVJCwLvt8uWmiRdg_o
title: glove-300d_es.vec, id: 167op9mvIKVfH1XGm3dkZ7afHReiqvmIN
title: glove.6B.300d_en.txt, id: 1FRfAM3GouOxelBo_gwj5n4YNWDMP6CkL
title: wiki_fasttext_es.vec, id: 1S1ObfL0R15IzRK_T1RrSxZMn0TJDECsN
title: wiki_fasttext_en.vec, id: 18zPA_9tWmlNZChdfMKHTp0Ka1SQsnkj9
title: TaskA, id: 1Hx5OP1Yrlh37yYzSMtsv6Ui_fuzOuG04
title: TaskB, id: 1JsHac7_mxx_M8BapYSzcDKnzl4M9e1Yb


In [0]:
#we = {'id': '1FRfAM3GouOxelBo_gwj5n4YNWDMP6CkL', 'file': 'glove.6B.300d_en.txt'}
we = {'id': '18zPA_9tWmlNZChdfMKHTp0Ka1SQsnkj9', 'file': 'wiki_fasttext_en.vec'}

In [0]:
word_vector = drive.CreateFile({'id': we['id']})
word_vector.GetContentFile(we['file'])

In [0]:
def word_embeddings(word_index, num_words, word_embedding_dim):
    embeddings_index = {}
    
    f = open(we['file'], 'r', encoding='utf-8')
    
    for line in tqdm(f):
        values = line.rstrip().rsplit(' ')
        word = values[0]
        coefs = np.asarray(values[1:], dtype='float32')
        embeddings_index[word] = coefs
    
    f.close()

    matrix = np.zeros((num_words, word_embedding_dim))
    
    for word, i in word_index.items():
        if i >= max_features:
            continue
        
        embedding_vector = embeddings_index.get(word)
        
        if embedding_vector is not None:
            matrix[i] = embedding_vector

    return matrix

In [0]:
embedding_dim = 300
max_features = 25000
maxlen = 100
batch_size = 32
epochs = 5
filter_sizes = [3,4]
num_filters = 512
drop = 0.5

def code_data(train_text, test_text, maxlen, max_features):
    data = np.concatenate((train_text, test_text), axis=0)

    # Treina um tokenizaddor nos dados de treino
    tokenizer = Tokenizer(num_words=max_features)
    tokenizer.fit_on_texts(data)

    # Tokeniza os dados
    X = tokenizer.texts_to_sequences(data)
    Y = tokenizer.texts_to_sequences(test_text)

    tweets = sequence.pad_sequences(X, maxlen=maxlen)
    x_test = sequence.pad_sequences(Y, maxlen=maxlen)
    
    word_index = tokenizer.word_index
    
    return tweets, x_test, word_index

In [0]:
tweets, x_test, word_index = code_data(train_text, test_text, maxlen, max_features)

In [0]:
num_words = min(max_features, len(word_index) + 1)

In [0]:
from tqdm import tqdm
embedding_matrix = word_embeddings(word_index, num_words, embedding_dim)

400000it [00:34, 11532.19it/s]


In [0]:
# Fase de classificação

tweet_input = Input(shape=(maxlen,), dtype='int32')

embedding = Embedding(num_words, embedding_dim, weights=[embedding_matrix], input_length=maxlen, trainable=True)(tweet_input)

reshape = Reshape((maxlen, embedding_dim, 1))(embedding)

cnn1 = Conv2D(num_filters, kernel_size=(filter_sizes[0], embedding_dim), padding='valid', kernel_initializer='normal', activation='tanh')(reshape)

max1 = MaxPool2D(pool_size=(maxlen - filter_sizes[0] + 1, 1), strides=(1,1), padding='valid')(cnn1)

cnn2 = Conv2D(num_filters, kernel_size=(filter_sizes[1], embedding_dim), padding='valid', kernel_initializer='normal', activation='tanh')(reshape)

max2 = MaxPool2D(pool_size=(maxlen - filter_sizes[1] + 1, 1), strides=(1,1), padding='valid')(cnn2)

concatenated_tensor = Concatenate(axis=1)([max1, max2])

flatten = Flatten()(concatenated_tensor)

dropout = Dropout(drop)(flatten)

dens = Dense(num_filters, activation='relu')(dropout)

output = Dense(1, activation='sigmoid')(dens)

model = Model(inputs=tweet_input, outputs=output)

opt = RMSprop(lr=0.001, rho=0.9, epsilon=None, decay=0.01)

model.compile(loss='binary_crossentropy', optimizer=opt, metrics=['accuracy'])

In [0]:
# Treinamento e predição - Hate Speech (HS)

classes = np.concatenate((hs_train, hs_test), axis=0)

x_train, x_val, y_train, y_val = train_test_split(tweets, classes, test_size=0.25, random_state=None)

model.summary()

model.fit(x_train, y_train, batch_size=batch_size, epochs=epochs, shuffle=True, validation_data=(x_val, y_val))

hs = (model.predict(x_test, batch_size=batch_size) > .5).astype(int)

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 100)          0                                            
__________________________________________________________________________________________________
embedding_1 (Embedding)         (None, 100, 300)     4993200     input_1[0][0]                    
__________________________________________________________________________________________________
reshape_1 (Reshape)             (None, 100, 300, 1)  0           embedding_1[0][0]                
__________________________________________________________________________________________________
conv2d_1 (Conv2D)               (None, 98, 1, 512)   461312      reshape_1[0][0]                  
__________________________________________________________________________________________________
conv2d_2 (

In [0]:
# Treinamento e predição - Target Range (TR)

classes = np.concatenate((tr_train, tr_test), axis=0)

x_train, x_val, y_train, y_val = train_test_split(tweets, classes, test_size=0.25, random_state=None)

model.summary()

model.fit(x_train, y_train, batch_size=batch_size, epochs=epochs, shuffle=True, validation_data=(x_val, y_val), validation_steps=None)

tr = (model.predict(x_test, batch_size=batch_size) > .5).astype(int)

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 100)          0                                            
__________________________________________________________________________________________________
embedding_1 (Embedding)         (None, 100, 300)     4993200     input_1[0][0]                    
__________________________________________________________________________________________________
reshape_1 (Reshape)             (None, 100, 300, 1)  0           embedding_1[0][0]                
__________________________________________________________________________________________________
conv2d_1 (Conv2D)               (None, 98, 1, 512)   461312      reshape_1[0][0]                  
__________________________________________________________________________________________________
conv2d_2 (

In [0]:
# Treinamento e predição - Aggressiveness (AG)

classes = np.concatenate((ag_train, ag_test), axis=0)

x_train, x_val, y_train, y_val = train_test_split(tweets, classes, test_size=0.25, random_state=None)

model.summary()

model.fit(x_train, y_train, batch_size=batch_size, epochs=epochs, shuffle=True, validation_data=(x_val, y_val), validation_steps=None)

ag = (model.predict(x_test, batch_size=batch_size) > .5).astype(int)

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 100)          0                                            
__________________________________________________________________________________________________
embedding_1 (Embedding)         (None, 100, 300)     4993200     input_1[0][0]                    
__________________________________________________________________________________________________
reshape_1 (Reshape)             (None, 100, 300, 1)  0           embedding_1[0][0]                
__________________________________________________________________________________________________
conv2d_1 (Conv2D)               (None, 98, 1, 512)   461312      reshape_1[0][0]                  
__________________________________________________________________________________________________
conv2d_2 (

In [0]:
def func(pred):
    output = []
    
    for array in pred:
        output.append(array[0])
    
    return output


hs_pred = func(hs)
tr_pred = func(tr)
ag_pred = func(ag)

print(len(hs_pred), len(tr_pred), len(ag_pred))

1000 1000 1000


In [0]:
from google.colab import files

with open("en_b.tsv", "w") as file:
    for i in range(len(hs_pred)):
        file.write(str(id_test[i]))
        file.write('\t')
        file.write(str(hs_pred[i]))
        file.write('\t')
        file.write(str(tr_pred[i]))
        file.write('\t')
        file.write(str(ag_pred[i]))
        file.write('\n')
    
files.download('en_b.tsv')